In [1]:
import pandas as pd
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json
import torch
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import regularizers
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

import itertools
import torch
import statsmodels.api as sm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error,mean_squared_log_error
from torch import nn
%matplotlib inline
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.preprocessing import MinMaxScaler

from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import regularizers

news_df = pd.read_csv("AstraZeneca PLC _ Search.csv",parse_dates=['Time'], dayfirst=False)

In [2]:
# Sort the DataFrame by 'Time'
news_df_sorted = news_df.sort_values(by='Time')

# Remove duplicate rows and select only 'Title' and 'Time' columns
news_df_cleaned = news_df_sorted[['Title', 'Time']].drop_duplicates()

# Rename the 'Time' column to 'Date'
news_df_cleaned = news_df_cleaned.rename(columns={'Time': 'Date'})

# Optionally, reset the index after removing duplicates
news_df_cleaned = news_df_cleaned.reset_index(drop=True)

# Save the cleaned data as JSON
news_df_cleaned.to_json('news_title.json', orient='records', lines=True)

# Display the cleaned DataFrame
news_df_cleaned.head()


,Title,Date
0,Cable backtracks on move to protect science in...,2014-07-08
1,Myners must give independent advice on indepen...,2014-07-09
2,AbbVie retraction shows panel has teeth,2014-07-09
3,Five examples of the UK Takeover Code in action,2014-07-09
4,UK Takeover Panel forces AbbVie to retract cla...,2014-07-09


In [3]:
# Determine the start and end dates from the news data
start_date = news_df_cleaned['Date'].min().strftime('%Y-%m-%d')
end_date = news_df_cleaned['Date'].max().strftime('%Y-%m-%d')

# Download historical data for AstraZeneca (AZN) based on the date range from the news data
stock_data = yf.download('AZN', start=start_date, end=end_date)

# Display the first few rows of the data
print(stock_data.head())

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2014-07-08  37.619999  37.630001  37.200001  37.334999  26.559191  2960800
2014-07-09  37.320000  37.540001  37.180000  37.520000  26.690794  5118000
2014-07-10  37.064999  37.389999  37.040001  37.360001  26.576975  2045000
2014-07-11  37.145000  37.325001  37.125000  37.255001  26.502283  1231000
2014-07-14  37.345001  37.494999  37.285000  37.430000  26.626774  2442600


In [4]:
from textblob import TextBlob

# Define a function to compute sentiment score
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns sentiment polarity (-1 to 1)

# Apply the function to the 'News' column
news_df_cleaned['Sentiment'] = news_df_cleaned['Title'].apply(get_sentiment)

In [5]:
news_df_cleaned

,Title,Date,Sentiment
0,Cable backtracks on move to protect science in...,2014-07-08,0.00
1,Myners must give independent advice on indepen...,2014-07-09,0.00
2,AbbVie retraction shows panel has teeth,2014-07-09,0.00
3,Five examples of the UK Takeover Code in action,2014-07-09,0.10
4,UK Takeover Panel forces AbbVie to retract cla...,2014-07-09,0.00
...,...,...,...
995,AstraZeneca calls on UK to change rules on inn...,2024-07-25,0.50
996,NHS watchdog blocks AstraZeneca breast cancer ...,2024-07-29,0.00
997,News updates from November 23: US to release 5...,NaT,0.00
998,Coronavirus: US automakers say vaccinated work...,NaT,0.00


In [6]:
modelName = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSequenceClassification.from_pretrained(modelName)
def calculateDailySentiment(headlines):
    texts = [headline['heading'] for headline in headlines]
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512, return_attention_mask=True)
    outputs = model(**inputs)
    logits = outputs.logits
    scores = logits.softmax(dim=1)
    averageScore = scores.mean(dim=0).tolist()
    return averageScore
#enddef


def analyzeAndSaveSentiment(inputFile, outputFile):
    result = {}
    
    # Open the file and process each line as a separate JSON object
    with open(inputFile, 'r',  encoding="utf8") as file:
        for line in file:
            # Parse each line (record) as a separate JSON object
            record = json.loads(line)
            date = record.get('Date')
            title = record.get('Title')

            if date and title:
                # Convert the timestamp to a readable date
                readable_date = datetime.fromtimestamp(int(date) / 1000).strftime('%Y-%m-%d')
                
                # Prepare the headlines in the required format
                headlines = [{'heading': title}]
                averageScore = calculateDailySentiment(headlines)
                print(f"{readable_date} > {averageScore}")
                result[readable_date] = averageScore
            #endif
        #endfor
    #endwith

    # Write the result to an output file
    with open(outputFile, 'w', encoding="utf8") as outFile:
        json.dump(result, outFile, indent=2)
    #endwith
#enddef

In [7]:
inputJsonFile = 'news_title.json'
outputJsonFile = 'news_sentimentscore.json'

analyzeAndSaveSentiment(inputJsonFile, outputJsonFile)

2014-07-08 > [0.030178336426615715, 0.03422543406486511, 0.10136785358190536, 0.3325415253639221, 0.5016868710517883]
2014-07-09 > [0.10291502624750137, 0.11560479551553726, 0.24775591492652893, 0.27704188227653503, 0.2566823959350586]
2014-07-09 > [0.35338830947875977, 0.3206016719341278, 0.2351720929145813, 0.06802640855312347, 0.022811438888311386]
2014-07-09 > [0.014788826927542686, 0.007935401052236557, 0.019648782908916473, 0.07863377779722214, 0.8789931535720825]
2014-07-09 > [0.45592352747917175, 0.20444481074810028, 0.16970273852348328, 0.11029187589883804, 0.059637103229761124]
2014-07-13 > [0.18320709466934204, 0.19196617603302002, 0.24269825220108032, 0.22939591109752655, 0.15273258090019226]
2014-07-14 > [0.15694545209407806, 0.17992740869522095, 0.21726123988628387, 0.2415277361869812, 0.2043382078409195]
2014-07-14 > [0.3743494153022766, 0.21798209846019745, 0.1857198178768158, 0.1152639165520668, 0.10668471455574036]
2014-07-14 > [0.032181479036808014, 0.038632638752460

2014-10-15 > [0.12542229890823364, 0.18764498829841614, 0.28012898564338684, 0.2647217810153961, 0.14208197593688965]
2014-10-15 > [0.6057943105697632, 0.22111466526985168, 0.1074722558259964, 0.03446544334292412, 0.031153328716754913]
2014-10-17 > [0.728945255279541, 0.1642584353685379, 0.06771066784858704, 0.02172105386853218, 0.017364567145705223]
2014-10-19 > [0.5825939774513245, 0.27900370955467224, 0.11408577859401703, 0.014876783825457096, 0.00943982694298029]
2014-10-24 > [0.007037394214421511, 0.008101463317871094, 0.04059244319796562, 0.2599543631076813, 0.6843143701553345]
2014-10-28 > [0.009538923390209675, 0.013573488220572472, 0.07718197256326675, 0.3800409138202667, 0.5196647047996521]
2014-11-04 > [0.27915358543395996, 0.18310315907001495, 0.17428292334079742, 0.1787615865468979, 0.1846986711025238]
2014-11-04 > [0.1759728640317917, 0.15218932926654816, 0.21560224890708923, 0.26412826776504517, 0.19210736453533173]
2014-11-06 > [0.16206853091716766, 0.10979743301868439,

2015-04-26 > [0.22961263358592987, 0.14036542177200317, 0.19320033490657806, 0.22257095575332642, 0.21425072848796844]
2015-04-26 > [0.4108641743659973, 0.21913759410381317, 0.19714003801345825, 0.10761768370866776, 0.06524060666561127]
2015-04-26 > [0.33962392807006836, 0.2624838948249817, 0.1552528440952301, 0.11439883708953857, 0.12824054062366486]
2015-04-27 > [0.30559125542640686, 0.22922079265117645, 0.23435738682746887, 0.12536250054836273, 0.10546799749135971]
2015-04-28 > [0.29887881875038147, 0.2350585162639618, 0.21721652150154114, 0.13105691969394684, 0.11778919398784637]
2015-05-05 > [0.4717194139957428, 0.27510762214660645, 0.12214164435863495, 0.059311408549547195, 0.07171989232301712]
2015-05-08 > [0.28426358103752136, 0.1543688327074051, 0.18921849131584167, 0.1707915961742401, 0.20135754346847534]
2015-05-15 > [0.008434281684458256, 0.009236328303813934, 0.039290349930524826, 0.2718704640865326, 0.6711685061454773]
2015-05-17 > [0.18887029588222504, 0.1487406492233276

2015-11-22 > [0.06227238476276398, 0.07906080037355423, 0.18749524652957916, 0.3302408456802368, 0.3409307301044464]
2015-11-23 > [0.5184769034385681, 0.2062521129846573, 0.128248393535614, 0.07137224078178406, 0.07565026730298996]
2015-11-24 > [0.09930920600891113, 0.067754827439785, 0.12067697197198868, 0.28065353631973267, 0.4316054582595825]
2015-11-26 > [0.3245774209499359, 0.25245174765586853, 0.19033703207969666, 0.12760405242443085, 0.10502970218658447]
2015-11-26 > [0.26748138666152954, 0.21174144744873047, 0.15871044993400574, 0.16242991387844086, 0.19963684678077698]
2015-12-02 > [0.047711391001939774, 0.04915965721011162, 0.13367053866386414, 0.3210163414478302, 0.4484419822692871]
2015-12-03 > [0.3178803622722626, 0.1710294634103775, 0.13396002352237701, 0.16093260049819946, 0.21619753539562225]
2015-12-04 > [0.20676152408123016, 0.23058649897575378, 0.23243147134780884, 0.17784783244132996, 0.15237265825271606]
2015-12-06 > [0.07330859452486038, 0.26067665219306946, 0.439

2016-04-06 > [0.7533055543899536, 0.18559686839580536, 0.051059771329164505, 0.006646269001066685, 0.003391472389921546]
2016-04-06 > [0.11566826701164246, 0.11265602707862854, 0.2528493404388428, 0.2709813416004181, 0.24784505367279053]
2016-04-06 > [0.32188674807548523, 0.17597921192646027, 0.15674962103366852, 0.15172745287418365, 0.1936570405960083]
2016-04-14 > [0.5464555025100708, 0.2282547652721405, 0.13926362991333008, 0.050416190177202225, 0.03560991585254669]
2016-04-19 > [0.032337430864572525, 0.027603330090641975, 0.05830616131424904, 0.23999252915382385, 0.6417605876922607]
2016-04-21 > [0.029074665158987045, 0.04375801980495453, 0.15340468287467957, 0.41200265288352966, 0.3617599606513977]
2016-04-25 > [0.16935519874095917, 0.11034587025642395, 0.19409507513046265, 0.24751733243465424, 0.2786865830421448]
2016-04-25 > [0.12545278668403625, 0.13952955603599548, 0.2589173913002014, 0.26742449402809143, 0.2086758017539978]
2016-04-25 > [0.2492520809173584, 0.2719348669052124

...

2021-01-26 > [0.4889059364795685, 0.19711525738239288, 0.15337568521499634, 0.09519033133983612, 0.06541267037391663]
2021-01-27 > [0.41013020277023315, 0.21682721376419067, 0.1954718828201294, 0.11714033037424088, 0.06043041869997978]
2021-01-28 > [0.45266586542129517, 0.24101442098617554, 0.18825913965702057, 0.07755075395107269, 0.040509872138500214]
2021-01-28 > [0.2461497187614441, 0.1301926225423813, 0.14060570299625397, 0.1927211433649063, 0.29033082723617554]
2021-01-28 > [0.2845010459423065, 0.18684297800064087, 0.18845131993293762, 0.16236604750156403, 0.17783859372138977]
2021-01-28 > [0.2736187279224396, 0.18988117575645447, 0.21181422472000122, 0.19661781191825867, 0.12806807458400726]
2021-01-28 > [0.31243330240249634, 0.24801181256771088, 0.20111402869224548, 0.1436455249786377, 0.0947953313589096]
2021-01-28 > [0.3602939546108246, 0.32869330048561096, 0.18502436578273773, 0.08612977713346481, 0.03985863924026489]
2021-01-29 > [0.11671396344900131, 0.08699631690979004, 0

2021-03-21 > [0.10087615251541138, 0.08766353875398636, 0.18547122180461884, 0.3466522693634033, 0.27933675050735474]
2021-03-21 > [0.3469915986061096, 0.23933027684688568, 0.1677922010421753, 0.14031529426574707, 0.10557064414024353]
2021-03-22 > [0.12048748880624771, 0.1005168929696083, 0.12903915345668793, 0.2882426977157593, 0.3617137670516968]
2021-03-22 > [0.21230077743530273, 0.17254438996315002, 0.24012897908687592, 0.23984943330287933, 0.13517649471759796]
2021-03-23 > [0.41494640707969666, 0.25842970609664917, 0.18313661217689514, 0.0918162390589714, 0.05167105048894882]
2021-03-23 > [0.24957434833049774, 0.18157416582107544, 0.1819130927324295, 0.20909379422664642, 0.17784462869167328]
2021-03-23 > [0.37761393189430237, 0.29474055767059326, 0.19311338663101196, 0.08830186724662781, 0.04623030126094818]
2021-03-24 > [0.20836220681667328, 0.18676036596298218, 0.2579873502254486, 0.19501395523548126, 0.1518760770559311]
2021-03-25 > [0.2590084969997406, 0.26162394881248474, 0.2

2021-05-19 > [0.16756971180438995, 0.12033254653215408, 0.15177154541015625, 0.26129069924354553, 0.29903551936149597]
2021-05-21 > [0.16911464929580688, 0.1676173210144043, 0.2515009045600891, 0.2563614249229431, 0.15540577471256256]
2021-05-22 > [0.016221754252910614, 0.03289874643087387, 0.18474742770195007, 0.4681662321090698, 0.2979658842086792]
2021-05-24 > [0.009061671793460846, 0.007747522555291653, 0.024033784866333008, 0.10498682409524918, 0.8541702032089233]
2021-05-25 > [0.2701530456542969, 0.166259303689003, 0.18549735844135284, 0.17097951471805573, 0.2071106731891632]
2021-05-26 > [0.7189188003540039, 0.15590991079807281, 0.07398821413516998, 0.028914675116539, 0.02226833440363407]
2021-05-26 > [0.3837798833847046, 0.21902883052825928, 0.17244921624660492, 0.1297277957201004, 0.09501419961452484]
2021-05-28 > [0.13761159777641296, 0.1810360997915268, 0.27702581882476807, 0.24054069817066193, 0.16378572583198547]
2021-06-01 > [0.5717823505401611, 0.2781069874763489, 0.0903

2022-06-05 > [0.07360705733299255, 0.07544559985399246, 0.11653564125299454, 0.3248491585254669, 0.40956252813339233]
2022-06-13 > [0.05636000260710716, 0.04869856685400009, 0.15929535031318665, 0.39084935188293457, 0.3447967767715454]
2022-06-22 > [0.11719509959220886, 0.09040188044309616, 0.16781014204025269, 0.26483309268951416, 0.35975977778434753]
2022-07-01 > [0.059697944670915604, 0.0627589225769043, 0.1357300877571106, 0.3257941007614136, 0.4160189628601074]
2022-07-16 > [0.05620427429676056, 0.07618480175733566, 0.19796472787857056, 0.37090620398521423, 0.298740029335022]
2022-07-19 > [0.025173749774694443, 0.0521354004740715, 0.17289113998413086, 0.41202831268310547, 0.33777135610580444]
2022-07-29 > [0.08718087524175644, 0.09857860952615738, 0.1453429013490677, 0.3213701844215393, 0.347527414560318]
2022-07-30 > [0.43739786744117737, 0.2450411468744278, 0.14495521783828735, 0.08708026260137558, 0.08552557229995728]
2022-08-16 > [0.3703545928001404, 0.24051879346370697, 0.206

2024-04-11 > [0.5408162474632263, 0.17820896208286285, 0.13051974773406982, 0.08152454346418381, 0.068930484354496]
2024-04-14 > [0.11171003431081772, 0.21321339905261993, 0.36474597454071045, 0.21143151819705963, 0.09889908879995346]
2024-04-14 > [0.8411708474159241, 0.09109902381896973, 0.03624598681926727, 0.015428062528371811, 0.016056234017014503]
2024-04-22 > [0.34511542320251465, 0.24019676446914673, 0.18489816784858704, 0.12759923934936523, 0.10219047218561172]
2024-04-25 > [0.3570277988910675, 0.25160205364227295, 0.16841231286525726, 0.10802976042032242, 0.11492803692817688]
2024-05-01 > [0.7987797260284424, 0.12535621225833893, 0.04133222624659538, 0.016931425780057907, 0.01760045439004898]
2024-05-01 > [0.4742334187030792, 0.20818720757961273, 0.15523405373096466, 0.09994552284479141, 0.062399763613939285]
2024-05-08 > [0.671649158000946, 0.1936941295862198, 0.084479421377182, 0.029288165271282196, 0.0208890363574028]
2024-05-20 > [0.18202538788318634, 0.10576513409614563, 

In [8]:
# Load the sentiment scores from the JSON file
with open('news_sentimentscore.json', 'r') as f:
    sentiment_data = json.load(f)

# Create a DataFrame to store the sentiment scores without averaging them
processed_data = {
    'Date': [],
    'SentimentScores': []
}

# Iterate through the JSON data and append the sentiment scores for each date
for date, scores in sentiment_data.items():
    processed_data['Date'].append(date)
    processed_data['SentimentScores'].append(scores)  # Keep the list of sentiment scores

# Convert processed data to a DataFrame
news_sentiment_df = pd.DataFrame(processed_data)

# Ensure the 'Date' column is in datetime format
news_sentiment_df['Date'] = pd.to_datetime(news_sentiment_df['Date'])

# Display the DataFrame
news_sentiment_df


,Date,SentimentScores
0,2014-07-08,"[0.030178336426615715, 0.03422543406486511, 0...."
1,2014-07-09,"[0.45592352747917175, 0.20444481074810028, 0.1..."
2,2014-07-13,"[0.18320709466934204, 0.19196617603302002, 0.2..."
3,2014-07-14,"[0.032181479036808014, 0.03863263875246048, 0...."
4,2014-07-15,"[0.040425244718790054, 0.033995620906353, 0.08..."
...,...,...
683,2024-06-14,"[0.22269688546657562, 0.1097579151391983, 0.16..."
684,2024-06-24,"[0.04238286241889, 0.06855291873216629, 0.2035..."
685,2024-07-24,"[0.18485687673091888, 0.18940424919128418, 0.1..."
686,2024-07-25,"[0.16694074869155884, 0.16756229102611542, 0.2..."


In [9]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-07-08,37.619999,37.630001,37.200001,37.334999,26.559191,2960800
2014-07-09,37.320000,37.540001,37.180000,37.520000,26.690794,5118000
2014-07-10,37.064999,37.389999,37.040001,37.360001,26.576975,2045000
2014-07-11,37.145000,37.325001,37.125000,37.255001,26.502283,1231000
2014-07-14,37.345001,37.494999,37.285000,37.430000,26.626774,2442600
...,...,...,...,...,...,...
2024-07-22,79.620003,79.809998,78.970001,79.160004,78.676437,3521000
2024-07-23,78.900002,79.199997,78.500000,79.169998,78.686371,2965100
2024-07-24,78.550003,79.769997,78.550003,79.709999,79.223068,4581100


In [10]:
news_sentiment_df.head()

,Date,SentimentScores
0,2014-07-08,"[0.030178336426615715, 0.03422543406486511, 0...."
1,2014-07-09,"[0.45592352747917175, 0.20444481074810028, 0.1..."
2,2014-07-13,"[0.18320709466934204, 0.19196617603302002, 0.2..."
3,2014-07-14,"[0.032181479036808014, 0.03863263875246048, 0...."
4,2014-07-15,"[0.040425244718790054, 0.033995620906353, 0.08..."


In [11]:
# Ensure the index of stock_data is converted to datetime if not already done
stock_data.index = pd.to_datetime(stock_data.index)

# Reset the index of stock_data to turn 'Date' into a regular column
stock_data = stock_data.reset_index()

# Merge the two DataFrames on the 'Date' column
dfMerged = pd.merge(stock_data, news_sentiment_df, on='Date', how='left')

# Check if each element in 'SentimentScores' is a list, and replace invalid ones with NaN or an empty list
dfMerged['SentimentScores'] = dfMerged['SentimentScores'].apply(lambda x: x if isinstance(x, list) else [None] * 5)

# Ensure each list in 'SentimentScores' has 5 elements (or adjust this if your data varies)
dfMerged['SentimentScores'] = dfMerged['SentimentScores'].apply(lambda x: x if len(x) == 5 else [None] * 5)

# Convert the 'SentimentScores' list into separate columns
dfSentimentColumns = pd.DataFrame(dfMerged['SentimentScores'].tolist(), columns=['feature1', 'feature2', 'feature3', 'feature4', 'feature5'])

# Concatenate the origi|nal DataFrame with the new sentiment columns
dfMerged = pd.concat([dfMerged, dfSentimentColumns], axis=1)

# Drop unnecessary columns
dfMerged = dfMerged.drop(columns=['SentimentScores'])

# Display the merged DataFrame
dfMerged.head()


,Date,Open,High,Low,Close,Adj Close,Volume,feature1,feature2,feature3,feature4,feature5
0,2014-07-08,37.619999,37.630001,37.200001,37.334999,26.559191,2960800,0.030178,0.034225,0.101368,0.332542,0.501687
1,2014-07-09,37.320000,37.540001,37.180000,37.520000,26.690794,5118000,0.455924,0.204445,0.169703,0.110292,0.059637
2,2014-07-10,37.064999,37.389999,37.040001,37.360001,26.576975,2045000,NaN,NaN,NaN,NaN,NaN
3,2014-07-11,37.145000,37.325001,37.125000,37.255001,26.502283,1231000,NaN,NaN,NaN,NaN,NaN
4,2014-07-14,37.345001,37.494999,37.285000,37.430000,26.626774,2442600,0.032181,0.038633,0.154576,0.391026,0.383585


In [12]:
dfMerged.fillna(method='ffill', inplace=True)


In [13]:
dfMerged

,Date,Open,High,Low,Close,Adj Close,Volume,feature1,feature2,feature3,feature4,feature5
0,2014-07-08,37.619999,37.630001,37.200001,37.334999,26.559191,2960800,0.030178,0.034225,0.101368,0.332542,0.501687
1,2014-07-09,37.320000,37.540001,37.180000,37.520000,26.690794,5118000,0.455924,0.204445,0.169703,0.110292,0.059637
2,2014-07-10,37.064999,37.389999,37.040001,37.360001,26.576975,2045000,0.455924,0.204445,0.169703,0.110292,0.059637
3,2014-07-11,37.145000,37.325001,37.125000,37.255001,26.502283,1231000,0.455924,0.204445,0.169703,0.110292,0.059637
4,2014-07-14,37.345001,37.494999,37.285000,37.430000,26.626774,2442600,0.032181,0.038633,0.154576,0.391026,0.383585
...,...,...,...,...,...,...,...,...,...,...,...,...
2526,2024-07-22,79.620003,79.809998,78.970001,79.160004,78.676437,3521000,0.042383,0.068553,0.203528,0.402966,0.282570
2527,2024-07-23,78.900002,79.199997,78.500000,79.169998,78.686371,2965100,0.042383,0.068553,0.203528,0.402966,0.282570
2528,2024-07-24,78.550003,79.769997,78.550003,79.709999,79.223068,4581100,0.184857,0.189404,0.181431,0.179348,0.264961
2529,2024-07-25,76.639999,78.660004,76.459999,78.519997,78.040337,10636000,0.166941,0.167562,0.228300,0.234898,0.202299


In [14]:
def createDataset(dataset, target, lookBack=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - lookBack):
        a = dataset[i:(i + lookBack), :]
        dataX.append(a)
        dataY.append(target[i + lookBack])
    return np.array(dataX), np.array(dataY)
#enddef

In [15]:
features = dfMerged.drop(['Date', 'Close'], axis=1).values
target = dfMerged['Close'].values

scalerFeatures = MinMaxScaler(feature_range=(0, 1))
scalerTarget = MinMaxScaler(feature_range=(0, 1))

featuresScaled = scalerFeatures.fit_transform(features)
targetScaled = scalerTarget.fit_transform(target.reshape(-1, 1))

lookBack = 60
X, y = createDataset(featuresScaled, targetScaled, lookBack)
print(X[:2])
print(y[:2])

trainSize = int(len(X) * 0.8)
testSize = len(X) - trainSize
trainX, testX = X[0:trainSize, :], X[trainSize:len(X), :]
trainY, testY = y[0:trainSize], y[trainSize:len(y)]

trainX = np.reshape(trainX, (trainX.shape[0], lookBack, trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], lookBack, testX.shape[2]))

[[[0.21356736 0.21141919 0.21216539 ... 0.21275771 0.68999684 0.57602104]
  [0.20808191 0.20977745 0.21180115 ... 0.3792027  0.21831613 0.06513549]
  [0.20341925 0.2070412  0.20925153 ... 0.3792027  0.21831613 0.06513549]
  ...
  [0.18129456 0.18287117 0.18411948 ... 0.35021531 0.5648871  0.40779189]
  [0.17864325 0.18478653 0.18384632 ... 0.38670364 0.78123052 0.43468846]
  [0.18156882 0.1842393  0.18430157 ... 0.22476745 0.10628387 0.0489919 ]]

 [[0.20808191 0.20977745 0.21180115 ... 0.3792027  0.21831613 0.06513549]
  [0.20341925 0.2070412  0.20925153 ... 0.3792027  0.21831613 0.06513549]
  [0.20488207 0.20585553 0.2107995  ... 0.3792027  0.21831613 0.06513549]
  ...
  [0.17864325 0.18478653 0.18384632 ... 0.38670364 0.78123052 0.43468846]
  [0.18156882 0.1842393  0.18430157 ... 0.22476745 0.10628387 0.0489919 ]
  [0.17690623 0.17493616 0.17437627 ... 0.22476745 0.10628387 0.0489919 ]]]
[[0.17184659]
 [0.16048709]]


In [ ]:
batchSize = 8
epoch = 88
neurons = 135
dropout = 0.15

model = Sequential()
model.add(LSTM(neurons, return_sequences=True, activation='tanh', input_shape=(lookBack, features.shape[1])))
model.add(Dropout(dropout))
model.add(LSTM(neurons, return_sequences=True, activation='tanh'))
model.add(Dropout(dropout))
model.add(LSTM(neurons, activation='tanh'))
model.add(Dropout(dropout))

model.add(Dense(units=1, activation='linear', activity_regularizer=regularizers.l1(0.00001)))
model.add(Activation('tanh'))
model.summary()

model.compile(loss='mean_squared_error' , optimizer='RMSprop')

model.fit(trainX, trainY, epochs=epoch, batch_size=batchSize, verbose=1, validation_split=0.2)

Model: "sequential_101"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_303 (LSTM)                      │ (None, 60, 135)             │          78,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_303 (Dropout)                │ (None, 60, 135)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_304 (LSTM)                      │ (None, 60, 135)             │         146,340 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_304 (Dropout)                │ (None, 60, 135)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_305 (LSTM)                      │ (None, 135)                 │         146,340 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_305 (Dropout)                │ (None, 135)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_101 (Dense)                    │ (None, 1)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 371,656 (1.42 MB)

 Trainable params: 371,656 (1.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - loss: 0.0119 - val_loss: 0.0037
Epoch 2/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 0.0016 - val_loss: 0.0114
Epoch 3/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - loss: 0.0011 - val_loss: 0.0021
Epoch 4/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 5/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 8.8285e-04 - val_loss: 0.0029
Epoch 6/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 8.3608e-04 - val_loss: 0.0017
Epoch 7/88
198/198 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 8.1509e-04 - val_loss: 0.0014
Epoch 8/88
153/198 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 7.5605e-04

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredictInv = scalerTarget.inverse_transform(trainPredict)
trainYInv = scalerTarget.inverse_transform(np.reshape(trainY, (trainY.shape[0], 1)))
testPredictInv = scalerTarget.inverse_transform(testPredict)
testYInv = scalerTarget.inverse_transform(np.reshape(testY, (testY.shape[0], 1)))

In [ ]:
trainScore = np.sqrt(mean_squared_error(trainYInv[:, 0], trainPredictInv[:, 0]))
print(f'Training RMSE: {trainScore}')
testScore = np.sqrt(mean_squared_error(testYInv[:, 0], testPredictInv[:, 0]))
print(f'Testing RMSE: {testScore}')

trainAccuracy = 100 - (trainScore / np.mean(trainYInv) * 100)
testAccuracy = 100 - (testScore / np.mean(testYInv) * 100)

print(f'Training Accuracy: {trainAccuracy:.2f}%')
print(f'Testing Accuracy: {testAccuracy:.2f}%')

In [ ]:
dates = dfMerged['Date'].values
sampleInterval = 60
sampledDates = dates[::sampleInterval]

plt.figure(figsize=(20, 10))
plt.plot(dates[:len(trainY)], trainY, label='Actual Train')
plt.plot(dates[:len(trainPredict)], trainPredict, label='Predicted Train')
plt.plot(dates[len(trainY):len(trainY) + len(testY)], testY, label='Actual Test')
plt.plot(dates[len(trainPredict):len(trainPredict) + len(testPredict)], testPredict, label='Predicted Test')

plt.xticks(sampledDates, rotation=90)

plt.legend()
plt.show()

In [ ]:
lstm_predictions = model.predict(testX)

testY_univariate = testY[:, 0]

# Flatten the LSTM predictions if necessary
lstm_predictions = lstm_predictions.squeeze()  # Shape will now be (495,)

# Calculate residuals between true values and predictions
residuals = testY_univariate - lstm_predictions  # Shape will be (495,)

In [ ]:
import pmdarima as pm
import numpy as np

# Assuming residuals are univariate, ensure they are in 1D
residuals = residuals.reshape(-1)  # Flatten if necessary

# Fit auto_arima model
auto_arima_model = pm.auto_arima(residuals, 
                                 start_p=0, start_q=0,       # Initial p and q values
                                 max_p=5, max_q=5,           # Maximum p and q values to explore
                                 d=None,                     # Let auto_arima detect the order of differencing (d)
                                 seasonal=False,             # Set False for non-seasonal data
                                 stepwise=True,              # Use stepwise search to reduce computation
                                 trace=True,                 # Show output of model fitting process
                                 error_action='ignore',      # Ignore if an order does not fit
                                 suppress_warnings=True,     # Suppress warnings for cleaner output
                                 maxiter=10)                 # Set a limit for iterations

# Summary of the chosen ARIMA model
print(auto_arima_model.summary())

# Forecast residuals for the length of the test set
arima_residual_predictions = auto_arima_model.predict(n_periods=len(testY))

# Check the predicted residuals
print(arima_residual_predictions.shape)
print(arima_residual_predictions[:5])


In [ ]:
# Fit ARIMA model to the residuals
import statsmodels.api as sm
arima_model = sm.tsa.arima.ARIMA(residuals, order=(0,0,0))  # Adjust order as needed
arima_fitted = arima_model.fit()

# Predict residuals using ARIMA
arima_residual_predictions = arima_fitted.forecast(steps=len(testX))

In [ ]:
hybrid_predictions = lstm_predictions + arima_residual_predictions


In [ ]:
dates = dfMerged['Date'].values
sampleInterval = 60
sampledDates = dates[::sampleInterval]

plt.figure(figsize=(20, 10))

# Plot actual training values
plt.plot(dates[:len(trainY)], trainY, label='Actual Train', color='blue')

# Plot predicted training values (LSTM only)
plt.plot(dates[:len(trainPredict)], trainPredict, label='Predicted Train (LSTM)', color='orange')

# Plot actual test values
plt.plot(dates[len(trainY):len(trainY) + len(testY)], testY, label='Actual Test', color='green')

# Plot predicted test values (LSTM only)
plt.plot(dates[len(trainPredict):len(trainPredict) + len(testPredict)], testPredict, label='Predicted Test (LSTM)', color='red')

# Plot predicted test values (LSTM-ARIMA hybrid)
plt.plot(dates[len(trainPredict):len(trainPredict) + len(hybrid_predictions)], 
         hybrid_predictions, label='Predicted Test (LSTM + ARIMA)', color='purple', linestyle='--')

# Set xticks for the sampled dates
plt.xticks(sampledDates, rotation=90)

# Add legend
plt.legend()

# Show plot
plt.show()


In [ ]:
from sklearn.metrics import r2_score
import numpy as np

# Calculate metrics for LSTM-only model
mse_lstm = mean_squared_error(testY, testPredict)
rmse_lstm = np.sqrt(mse_lstm)
mae_lstm = mean_absolute_error(testY, testPredict)
r2_lstm = r2_score(testY, testPredict)
mape_lstm = np.mean(np.abs((testY - testPredict) / testY)) * 100

# Calculate metrics for LSTM-ARIMA hybrid model
mse_hybrid = mean_squared_error(testY, hybrid_predictions)
rmse_hybrid = np.sqrt(mse_hybrid)
mae_hybrid = mean_absolute_error(testY, hybrid_predictions)
r2_hybrid = r2_score(testY, hybrid_predictions)
mape_hybrid = np.mean(np.abs((testY - hybrid_predictions) / testY)) * 100

# Print the results for LSTM-only model
print(f"LSTM-Only Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse_lstm}")
print(f"Root Mean Squared Error (RMSE): {rmse_lstm}")
print(f"Mean Absolute Error (MAE): {mae_lstm}")
print(f"R-squared (R²): {r2_lstm}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_lstm}%\n")

# Print the results for LSTM-ARIMA hybrid model
print(f"LSTM-ARIMA Hybrid Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse_hybrid}")
print(f"Root Mean Squared Error (RMSE): {rmse_hybrid}")
print(f"Mean Absolute Error (MAE): {mae_hybrid}")
print(f"R-squared (R²): {r2_hybrid}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_hybrid}%")


In [26]:
import optuna
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop

# Function to build the LSTM model
def create_model(neurons, dropout):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, activation='tanh', input_shape=(lookBack, features.shape[1])))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, return_sequences=True, activation='tanh'))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=RMSprop())
    return model

# Objective function for optimization
def objective(trial):
    # Suggest hyperparameters
    neurons = trial.suggest_int('neurons', 50, 150)          # Range for neurons
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)     # Range for dropout
    batch_size = trial.suggest_int('batch_size', 8, 64)      # Range for batch size
    epochs = trial.suggest_int('epochs', 20, 100)            # Range for epochs

    # Create and train the model
    model = create_model(neurons, dropout)
    history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    
    # Return validation loss as the metric to minimize
    val_loss = min(history.history['val_loss'])
    return val_loss

# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Best parameters found
best_params = study.best_params
print("Best parameters: ", best_params)


[I 2024-09-22 10:12:15,073] A new study created in memory with name: no-name-0d9c2415-496d-4c43-8a19-1cea2d142ea5
C:\Users\will\AppData\Local\Temp\ipykernel_14732\202567461.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.1, 0.5)     # Range for dropout
C:\Users\will\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2024-09-22 10:13:38,334] Trial 0 finished with value: 0.00107559934258461 and parameters: {'neurons': 150, 'dropout': 0.28580804135315807, 'batch_size': 41, 'epochs': 23}. Best is trial 0 with value: 0.00107559934258461.
[I 2024-09-22 10:15:35,183] Trial 1 finish

[I 2024-09-22 12:35:16,886] Trial 33 finished with value: 0.0008209124789573252 and parameters: {'neurons': 90, 'dropout': 0.20186393987346307, 'batch_size': 19, 'epochs': 51}. Best is trial 24 with value: 0.0005134977982379496.
[I 2024-09-22 12:41:31,036] Trial 34 finished with value: 0.0005935042863711715 and parameters: {'neurons': 79, 'dropout': 0.24321795137381036, 'batch_size': 11, 'epochs': 85}. Best is trial 24 with value: 0.0005134977982379496.
[I 2024-09-22 12:45:25,485] Trial 35 finished with value: 0.0007354039116762578 and parameters: {'neurons': 70, 'dropout': 0.3111710015858258, 'batch_size': 16, 'epochs': 72}. Best is trial 24 with value: 0.0005134977982379496.
[I 2024-09-22 12:52:15,565] Trial 36 finished with value: 0.0005229634116403759 and parameters: {'neurons': 104, 'dropout': 0.1468633555282508, 'batch_size': 8, 'epochs': 67}. Best is trial 24 with value: 0.0005134977982379496.
[I 2024-09-22 12:55:08,944] Trial 37 finished with value: 0.0007914130692370236 and pa

[I 2024-09-22 16:28:13,199] Trial 69 finished with value: 0.0006637116312049329 and parameters: {'neurons': 127, 'dropout': 0.30096316982592836, 'batch_size': 15, 'epochs': 97}. Best is trial 55 with value: 0.000501760805491358.
[I 2024-09-22 16:36:28,763] Trial 70 finished with value: 0.0005402197712101042 and parameters: {'neurons': 124, 'dropout': 0.27466024245427884, 'batch_size': 10, 'epochs': 100}. Best is trial 55 with value: 0.000501760805491358.
[I 2024-09-22 16:43:57,788] Trial 71 finished with value: 0.0005546066677197814 and parameters: {'neurons': 119, 'dropout': 0.27129404608282415, 'batch_size': 10, 'epochs': 94}. Best is trial 55 with value: 0.000501760805491358.
[I 2024-09-22 16:53:25,420] Trial 72 finished with value: 0.000570251198951155 and parameters: {'neurons': 130, 'dropout': 0.29015364938874216, 'batch_size': 12, 'epochs': 100}. Best is trial 55 with value: 0.000501760805491358.
[I 2024-09-22 17:02:46,927] Trial 73 finished with value: 0.0005690573016181588 and

Best parameters:  {'neurons': 135, 'dropout': 0.15398324549808712, 'batch_size': 8, 'epochs': 88}


In [ ]:
0.0005134977982379496.
0.000501760805491358.